### Reading csv files

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np


def csv2ArmIO(folder: Path, code=200):
    # Concatenate all .csv files
    print(sorted(folder.glob("*.csv")))
    dfs = [pd.read_csv(fp, sep=",") for fp in sorted(folder.glob("*.csv"))]
    data = pd.concat(dfs, ignore_index=True)
    data = data[(data["eventCode"] == code) & (data["chosenPort"].isin([1, 2]))]

    blockId = data["blockId"].to_numpy()
    block_starts = np.where(np.diff(blockId, prepend=-1) != 0, 1, 0)
    data["sessionId"] = np.cumsum(block_starts)

    # print(data["chosenPort"].unique())

    return data

### Example data

In [ ]:
basedir = Path("D:/Data/mab/BGdataset")
data_folder = basedir / "BGM0" / "raw_data"
task_df = csv2ArmIO(data_folder)
probs = task_df.loc[:, ["port1Prob", "port2Prob"]].to_numpy()
choices = task_df["chosenPort"].to_numpy()
ntrials_session = np.unique(task_df["sessionId"].to_numpy(), return_counts=True)
task_df

### Calculate performance over sessions

In [ ]:
is_choice_high = np.argmax(probs, axis=1) + 1 == choices

is_choice_high_per_session = pd.DataFrame(
    np.split(is_choice_high.astype(int), np.cumsum(ntrials_session)[:-1])
).to_numpy()

perf = np.nanmean(is_choice_high_per_session, axis=0)